In [848]:
import pandas as pd 
import geopandas as gpd 
import numpy as np 
import json 
from glob import glob 

import sys 
sys.path.append("../")
from logger import setup_logger
logger = setup_logger("analysis-df-assembly")
logger.setLevel("INFO")

WGS='EPSG:4326'
PROJ='EPSG:2263'

import os 

logger.info("Modules loaded.")



2024-10-23 16:51:43 - analysis-df-assembly - INFO - Modules loaded.


In [849]:
INTERNAL_DATA = True 

USE_SMOOTHING = True


In [850]:
ICAR_NONE_RUN='../runs/icar_none/simulated_False/ahl_True/20241021-1038'
ICAR_CHEATING_RUN='../runs/icar_cheating/simulated_False/ahl_True/20241022-1130'

In [851]:
ICAR_NONE_ESTIMATES = glob(f"{ICAR_NONE_RUN}/estimate*.csv")
ICAR_CHEATING_ESTIMATES = glob(f"{ICAR_CHEATING_RUN}/estimate*.csv")
logger.info(f"Found {len(ICAR_NONE_ESTIMATES)} ICAR_NONE estimates and {len(ICAR_CHEATING_ESTIMATES)} ICAR_CHEATING estimates.")

2024-10-23 16:51:43 - analysis-df-assembly - INFO - Found 2 ICAR_NONE estimates and 3 ICAR_CHEATING estimates.


In [852]:
icar_cheating_estimates = {} 
for f in ICAR_CHEATING_ESTIMATES:
    df = pd.read_csv(f)
    df['tract_id'] = df['tract_id'].astype(int).astype(str)
    icar_cheating_estimates[os.path.splitext(os.path.basename(f))[0]] = df


In [853]:
icar_none_estimates = {} 
for f in ICAR_NONE_ESTIMATES:
    df = pd.read_csv(f)
    df['tract_id'] = df['tract_id'].astype(int).astype(str)
    icar_none_estimates[os.path.splitext(os.path.basename(f)[0])] = df
    

In [854]:

if USE_SMOOTHING: 
    icar_model_estimates = icar_cheating_estimates
    logger.info("Using smoothed estimates.")
else:
    icar_model_estimates = icar_none_estimates
    logger.info("Using unsmoothed estimates.")

2024-10-23 16:51:43 - analysis-df-assembly - INFO - Using smoothed estimates.


In [855]:
ct_nyc = gpd.read_file('geo/data/ct-nyc-wi-2020.geojson')


TO_DROP = ['OBJECTID','BoroCode','CT2020','CDEligibil','NTA2020','CDTA2020','Shape__Area','Shape__Length','geometry']
ct_nyc.drop(columns=TO_DROP, inplace=True)

ct_nyc = ct_nyc.set_index('GEOID').astype(str)

logger.info(f"Loaded NYC CT shapefile with {len(ct_nyc.index)} CTs.")

2024-10-23 16:51:43 - analysis-df-assembly - INFO - Loaded NYC CT shapefile with 2327 CTs.


In [856]:
ct_nyc 

,CTLabel,BoroName,BoroCT2020,NTAName,CDTANAME,PUMA
GEOID,,,,,,
36061000100,1,Manhattan,1000100,The Battery-Governors Island-Ellis Island-Libe...,MN01 Financial District-Tribeca (CD 1 Equivalent),4121
36061000201,2.01,Manhattan,1000201,Chinatown-Two Bridges,MN03 Lower East Side-Chinatown (CD 3 Equivalent),4103
36061000600,6,Manhattan,1000600,Chinatown-Two Bridges,MN03 Lower East Side-Chinatown (CD 3 Equivalent),4103
36061001401,14.01,Manhattan,1001401,Lower East Side,MN03 Lower East Side-Chinatown (CD 3 Equivalent),4103
36061001402,14.02,Manhattan,1001402,Lower East Side,MN03 Lower East Side-Chinatown (CD 3 Equivalent),4103
...,...,...,...,...,...,...
36085017600,176,Staten Island,5017600,Annadale-Huguenot-Prince's Bay-Woodrow,SI03 South Shore (CD 3 Approximation),4503
36085022802,228.02,Staten Island,5022802,Freshkills Park (North),SI02 Mid-Island (CD 2 Approximation),4502
36085029102,291.02,Staten Island,5029102,New Springville-Willowbrook-Bulls Head-Travis,SI02 Mid-Island (CD 2 Approximation),4502


In [857]:
ct_nyc_clip = gpd.read_file('geo/data/ct-nyc-2020.geojson')
logger.info(f"Loaded NYC CT (water clipped) shapefile with {len(ct_nyc_clip.index)} CTs.")

2024-10-23 16:51:44 - analysis-df-assembly - INFO - Loaded NYC CT (water clipped) shapefile with 2325 CTs.


In [858]:
ct_nyc = ct_nyc.merge(icar_model_estimates['estimate_p_y'], left_index=True, right_on='tract_id', suffixes=('_ct', '_p_y')).set_index('tract_id')
ct_nyc = ct_nyc.merge(icar_model_estimates['estimate_at_least_one_positive_image_by_area'], left_index=True, right_on='tract_id', suffixes=('_ct', '_p_alop')).set_index('tract_id')
ct_nyc = ct_nyc.merge(icar_model_estimates['estimate_at_least_one_positive_image_by_area_if_you_have_100_images'], left_index=True, right_on='tract_id', suffixes=('_ct', '_p_alop_100')).set_index('tract_id')

# drop empirical_estimate_* cols 
TO_DROP = [c for c in ct_nyc.columns if 'empirical_estimate_' in c]
ct_nyc.drop(columns=TO_DROP, inplace=True)

logger.info(f"Merged NYC CT shapefile with icar model estimates.")

2024-10-23 16:51:44 - analysis-df-assembly - INFO - Merged NYC CT shapefile with icar model estimates.


In [859]:
# Load data
dp05_nyc_md = pd.read_json('demo/data/acs22_dp05_md.json')

# Normalize the 'variables' column in the JSON
dp05_nyc_md = pd.json_normalize(dp05_nyc_md['variables']).set_index(dp05_nyc_md.index)

# Parse out the 'label' column
# In all rows of the 'label', get the lowest and highest number of '!!'
min_sep = min(dp05_nyc_md['label'].apply(lambda x: x.count('!!')))
max_sep = max(dp05_nyc_md['label'].apply(lambda x: x.count('!!')))

# Create 'desc_i' columns for each level of '!!'
for i in range(min_sep + 1, max_sep + 2):  # Adjusting range to account for correct indexing
    dp05_nyc_md[f'desc_{i}'] = dp05_nyc_md['label'].apply(
        lambda x: x.split('!!')[i-1] if len(x.split('!!')) >= i else None
    )

# drop TO_DROP 
TO_DROP = ['label','concept','predicateType','group','limit','predicateOnly']
dp05_nyc_md = dp05_nyc_md.drop(columns=TO_DROP)

desc_1_filter = ['Estimate']
dp05_nyc_md = dp05_nyc_md[dp05_nyc_md['desc_1'].isin(desc_1_filter)]

# Output the modified dataframe
# display all rows 
dp05_nyc_md = dp05_nyc_md.sort_index()

# to csv 
dp05_nyc_md.to_csv('demo/data/acs22_dp05_md.csv')

In [860]:
dp05_nyc = pd.read_json('demo/data/acs22_dp05.json', orient='records')

dp05_nyc.columns = dp05_nyc.iloc[0]
dp05_nyc = dp05_nyc[1:]

dp05_nyc['tract_id'] = dp05_nyc['GEO_ID'].str.split('US', expand=True)[1]

RACE_COLS = {
    'DP05_0001E': 'total_population',
    'DP05_0079E': 'nhl_white_alone', 
    'DP05_0080E': 'nhl_black_alone', 
    'DP05_0073E': 'hispanic_alone', 
    'DP05_0082E': 'nhl_asian_alone'
}

race_nyc = dp05_nyc[list(RACE_COLS.keys())]
race_nyc.columns = race_nyc.columns.map(lambda x: RACE_COLS[x])
race_nyc.index = dp05_nyc['tract_id']
# cast all columns to ints 
race_nyc = race_nyc.astype(int)
race_nyc 

,total_population,nhl_white_alone,nhl_black_alone,hispanic_alone,nhl_asian_alone
tract_id,,,,,
36005000100,4446,1098,2000,1172,123
36005000200,4870,83,1281,3109,299
36005000400,6257,283,1559,4212,103
36005001600,6177,106,2132,3507,148
36005001901,2181,306,942,842,0
...,...,...,...,...,...
36085030302,6374,2209,1568,1625,918
36085031901,3674,289,1626,1469,224
36085031902,5053,473,2388,1913,217


In [861]:
ct_nyc = ct_nyc.merge(race_nyc, left_index=True, right_index=True)  

In [862]:
# Load data
s2801_nyc_md = pd.read_json('demo/data/acs22_s2801_md.json')

# Normalize the 'variables' column in the JSON
s2801_nyc_md = pd.json_normalize(s2801_nyc_md['variables']).set_index(s2801_nyc_md.index)

# Parse out the 'label' column
# In all rows of the 'label', get the lowest and highest number of '!!'
min_sep = min(s2801_nyc_md['label'].apply(lambda x: x.count('!!')))
max_sep = max(s2801_nyc_md['label'].apply(lambda x: x.count('!!')))

# Create 'desc_i' columns for each level of '!!'
for i in range(min_sep + 1, max_sep + 2):  # Adjusting range to account for correct indexing
    s2801_nyc_md[f'desc_{i}'] = s2801_nyc_md['label'].apply(
        lambda x: x.split('!!')[i-1] if len(x.split('!!')) >= i else None
    )

# drop TO_DROP 
TO_DROP = ['label','concept','predicateType','group','limit','predicateOnly']
s2801_nyc_md = s2801_nyc_md.drop(columns=TO_DROP)

desc_1_filter = ['Estimate']
s2801_nyc_md = s2801_nyc_md[s2801_nyc_md['desc_1'].isin(desc_1_filter)]

# Output the modified dataframe
# display all rows 
s2801_nyc_md = s2801_nyc_md.sort_index()

# to csv 
s2801_nyc_md.to_csv('demo/data/acs22_s2801_md.csv')

In [863]:
s2801_nyc = pd.read_json('demo/data/acs22_s2801.json', orient='records')

s2801_nyc.columns = s2801_nyc.iloc[0]
s2801_nyc = s2801_nyc[1:]

s2801_nyc['tract_id'] = s2801_nyc['GEO_ID'].str.split('US', expand=True)[1]

INTERNET_ACCESS_COLS = { 
    'S2801_C01_012E': 'num_households_with_internet',
    'S2801_C01_005E': 'num_households_with_smartphone'
}

internet_access_nyc = s2801_nyc[list(INTERNET_ACCESS_COLS.keys())]
internet_access_nyc.columns = internet_access_nyc.columns.map(lambda x: INTERNET_ACCESS_COLS[x])
internet_access_nyc.index = s2801_nyc['tract_id']
# cast all columns to ints
internet_access_nyc = internet_access_nyc.astype(int)
internet_access_nyc

,num_households_with_internet,num_households_with_smartphone
tract_id,,
36005000100,0,0
36005000200,1251,1384
36005000400,2264,2148
36005001600,1843,2035
36005001901,825,924
...,...,...
36085030302,1994,2029
36085031901,885,1078
36085031902,1294,1445


In [864]:
ct_nyc = ct_nyc.join(internet_access_nyc)

In [865]:
s1901_nyc_md = pd.read_json('demo/data/acs22_s1901_md.json')

# Normalize the 'variables' column in the JSON
s1901_nyc_md = pd.json_normalize(s1901_nyc_md['variables']).set_index(s1901_nyc_md.index)

# Parse out the 'label' column
# In all rows of the 'label', get the lowest and highest number of '!!'
min_sep = min(s1901_nyc_md['label'].apply(lambda x: x.count('!!')))
max_sep = max(s1901_nyc_md['label'].apply(lambda x: x.count('!!')))

# Create 'desc_i' columns for each level of '!!'
for i in range(min_sep + 1, max_sep + 2):  # Adjusting range to account for correct indexing
    s1901_nyc_md[f'desc_{i}'] = s1901_nyc_md['label'].apply(
        lambda x: x.split('!!')[i-1] if len(x.split('!!')) >= i else None
    )

# drop TO_DROP
TO_DROP = ['label','concept','predicateType','group','limit','predicateOnly']
s1901_nyc_md = s1901_nyc_md.drop(columns=TO_DROP)

desc_1_filter = ['Estimate']
s1901_nyc_md = s1901_nyc_md[s1901_nyc_md['desc_1'].isin(desc_1_filter)]

# Output the modified dataframe
# display all rows
s1901_nyc_md = s1901_nyc_md.sort_index()
s1901_nyc_md.to_csv('demo/data/acs22_s1901_md.csv')

In [866]:
s1901_nyc = pd.read_json('demo/data/acs22_s1901.json', orient='records')

s1901_nyc.columns = s1901_nyc.iloc[0]
s1901_nyc = s1901_nyc[1:]

s1901_nyc['tract_id'] = s1901_nyc['GEO_ID'].str.split('US', expand=True)[1]

INCOME_COLS = {
    'S1901_C01_012E': 'median_household_income'
}

income_nyc = s1901_nyc[list(INCOME_COLS.keys())]
income_nyc.columns = income_nyc.columns.map(lambda x: INCOME_COLS[x])
income_nyc.index = s1901_nyc['tract_id']

# cast all columns to ints
income_nyc = income_nyc.astype(int)
income_nyc

,median_household_income
tract_id,
36005000100,-666666666
36005000200,115064
36005000400,100553
36005001600,41362
36005001901,49500
...,...
36085030302,98535
36085031901,45942
36085031902,75057


In [867]:
ct_nyc = ct_nyc.join(income_nyc)

In [868]:
# Educational Attainment 
s1501_nyc_md = pd.read_json('demo/data/acs22_s1501_md.json')

# Normalize the 'variables' column in the JSON
s1501_nyc_md = pd.json_normalize(s1501_nyc_md['variables']).set_index(s1501_nyc_md.index)

# Parse out the 'label' column
# In all rows of the 'label', get the lowest and highest number of '!!'
min_sep = min(s1501_nyc_md['label'].apply(lambda x: x.count('!!')))
max_sep = max(s1501_nyc_md['label'].apply(lambda x: x.count('!!')))

# Create 'desc_i' columns for each level of '!!'
for i in range(min_sep + 1, max_sep + 2):  # Adjusting range to account for correct indexing
    s1501_nyc_md[f'desc_{i}'] = s1501_nyc_md['label'].apply(
        lambda x: x.split('!!')[i-1] if len(x.split('!!')) >= i else None
    )

# drop TO_DROP
TO_DROP = ['label','concept','predicateType','group','limit','predicateOnly']
s1501_nyc_md = s1501_nyc_md.drop(columns=TO_DROP)

desc_1_filter = ['Estimate']
s1501_nyc_md = s1501_nyc_md[s1501_nyc_md['desc_1'].isin(desc_1_filter)]

# Output the modified dataframe
# display all rows
s1501_nyc_md = s1501_nyc_md.sort_index()
s1501_nyc_md.to_csv('demo/data/acs22_s1501_md.csv')

In [869]:
s1501_nyc = pd.read_json('demo/data/acs22_s1501.json', orient='records')

s1501_nyc.columns = s1501_nyc.iloc[0]
s1501_nyc = s1501_nyc[1:]

s1501_nyc['tract_id'] = s1501_nyc['GEO_ID'].str.split('US', expand=True)[1]

EDUCATION_COLS = {
    'S1501_C01_009E': 'num_high_school_graduates',
    'S1501_C01_012E': 'num_bachelors_degree',
    'S1501_C01_013E': 'num_graduate_degree'
}

education_nyc = s1501_nyc[list(EDUCATION_COLS.keys())]
education_nyc.columns = education_nyc.columns.map(lambda x: EDUCATION_COLS[x])
education_nyc.index = s1501_nyc['tract_id']

# cast all columns to ints
education_nyc = education_nyc.astype(int)
education_nyc


,num_high_school_graduates,num_bachelors_degree,num_graduate_degree
tract_id,,,
36005000100,1835,34,41
36005000200,1089,489,671
36005000400,878,995,544
36005001600,1101,682,123
36005001901,408,283,164
...,...,...,...
36085030302,1357,1069,430
36085031901,549,185,175
36085031902,1020,677,241


In [870]:
# limited english speaking households 
s1602_nyc_md = pd.read_json('demo/data/acs22_s1602_md.json')

# Normalize the 'variables' column in the JSON
s1602_nyc_md = pd.json_normalize(s1602_nyc_md['variables']).set_index(s1602_nyc_md.index)

# Parse out the 'label' column
# In all rows of the 'label', get the lowest and highest number of '!!'
min_sep = min(s1602_nyc_md['label'].apply(lambda x: x.count('!!')))
max_sep = max(s1602_nyc_md['label'].apply(lambda x: x.count('!!')))

# Create 'desc_i' columns for each level of '!!'
for i in range(min_sep + 1, max_sep + 2):  # Adjusting range to account for correct indexing
    s1602_nyc_md[f'desc_{i}'] = s1602_nyc_md['label'].apply(
        lambda x: x.split('!!')[i-1] if len(x.split('!!')) >= i else None
    )

# drop TO_DROP
TO_DROP = ['label','concept','predicateType','group','limit','predicateOnly']
s1602_nyc_md = s1602_nyc_md.drop(columns=TO_DROP)

desc_1_filter = ['Estimate']

s1602_nyc_md = s1602_nyc_md[s1602_nyc_md['desc_1'].isin(desc_1_filter)]

# Output the modified dataframe
# display all rows
s1602_nyc_md = s1602_nyc_md.sort_index()
s1602_nyc_md.to_csv('demo/data/acs22_s1602_md.csv')



In [871]:
ct_nyc = ct_nyc.join(education_nyc)

In [872]:
s1602_nyc_md = pd.read_csv('demo/data/acs22_s1602_md.csv')

s1602_nyc = pd.read_json('demo/data/acs22_s1602.json', orient='records')

s1602_nyc.columns = s1602_nyc.iloc[0]
s1602_nyc = s1602_nyc[1:]

s1602_nyc['tract_id'] = s1602_nyc['GEO_ID'].str.split('US', expand=True)[1]

LEH_COLS = {
    'S1602_C03_001E': 'num_limited_english_speaking_households'
}

leh_nyc = s1602_nyc[list(LEH_COLS.keys())]
leh_nyc.columns = leh_nyc.columns.map(lambda x: LEH_COLS[x])
leh_nyc.index = s1602_nyc['tract_id']

# cast all columns to ints
leh_nyc = leh_nyc.astype(int)
leh_nyc



,num_limited_english_speaking_households
tract_id,
36005000100,0
36005000200,257
36005000400,445
36005001600,336
36005001901,55
...,...
36085030302,86
36085031901,46
36085031902,87


In [873]:
ct_nyc = ct_nyc.join(leh_nyc)

In [874]:
# TOPOLOGY 
topology_ct_nyc = pd.read_csv('geo/data/processed/ct_nyc_topology.csv', index_col=0)
topology_ct_nyc['GEOID'] = topology_ct_nyc['GEOID'].astype(str)
topology_ct_nyc = topology_ct_nyc.set_index('GEOID')
# prepend columns with 'ft_elevation'
topology_ct_nyc.columns = ['ft_elevation_' + c for c in topology_ct_nyc.columns]
ct_nyc = ct_nyc.merge(topology_ct_nyc, left_index=True, right_on='GEOID')

In [875]:
gdf_ct_nyc = gpd.read_file('geo/data/ct-nyc-wi-2020.geojson').to_crs(PROJ)[['GEOID','geometry']]


In [876]:
# basic geographic features 
gdf_ct_nyc['area'] = gdf_ct_nyc.area 
# add this col to ct_nyc
ct_nyc = ct_nyc.merge(gdf_ct_nyc[['GEOID','area']], left_index=True, right_on='GEOID').set_index('GEOID')


In [877]:
# FLOODNET 
floodnet_sensor = pd.read_csv('flooding/static/floodnet-flood-sensor-sep-2023.csv', engine='pyarrow')
floodnet_tide = pd.read_csv('flooding/static/floodnet-tide-sep-2023.csv', engine='pyarrow')
floodnet_weather = pd.read_csv('flooding/static/floodnet-weather-sep-2023.csv', engine='pyarrow')


all_floodnet_sensors_geo = pd.concat([floodnet_sensor.groupby('deployment_id').first()[['lat','lon']].reset_index(), floodnet_tide.groupby('sensor_id').first()[['lat','lon']].reset_index(), floodnet_weather.groupby('sensor_id').first()[['lat','lon']].reset_index()], axis=0)

all_floodnet_sensor_geo = gpd.GeoDataFrame(all_floodnet_sensors_geo, geometry=gpd.points_from_xy(all_floodnet_sensors_geo.lon, all_floodnet_sensors_geo.lat), crs='EPSG:4326').to_crs(2263)

del floodnet_sensor, floodnet_tide, floodnet_weather

logger.info("Loaded and processed Floodnet sensor data.")

2024-10-23 16:51:46 - analysis-df-assembly - INFO - Loaded and processed Floodnet sensor data.


In [878]:
# get the number of floodnet sensors in each census tract in gdf_ct_nyc, including tracts with 0 sensors
ct_nyc['n_sensors'] = gpd.sjoin(gdf_ct_nyc, all_floodnet_sensor_geo).groupby('GEOID').size().reindex(ct_nyc.index).fillna(0)

logger.info("Merged Floodnet sensor data with NYC CT shapefile.")
logger.info(ct_nyc['n_sensors'].describe().to_string())



2024-10-23 16:51:46 - analysis-df-assembly - INFO - Merged Floodnet sensor data with NYC CT shapefile.
2024-10-23 16:51:46 - analysis-df-assembly - INFO - count    2325.000000
mean        0.036989
std         0.274269
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         6.000000


In [879]:
# DEP STORMWATER 
dep_moderate = gpd.read_file('flooding/data/NYCFloodStormwaterFloodMaps/NYC Stormwater Flood Map - Moderate Flood (2.13 inches per hr) with Current Sea Levels/NYC_Stormwater_Flood_Map_Moderate_Flood_2_13_inches_per_hr_with_Current_Sea_Levels.gdb').to_crs(PROJ)

In [880]:
polygons = {}
for i, row in dep_moderate.iterrows():
    idx = 0
    for polygon in row['geometry'].geoms:
        polygons[f'{row["Flooding_Category"]}_{idx}'] = polygon
        idx += 1
    
# dataframe from dict
dep_moderate_flattened = gpd.GeoDataFrame(polygons, index=['geometry']).T

dep_moderate_flattened.set_geometry('geometry', inplace=True)
dep_moderate_flattened.crs = dep_moderate.crs

dep_moderate_flattened['Flooding_Category'] = dep_moderate_flattened.index.str.split('_').str[0].astype(int)

In [881]:
# get the total area of light and moderate flooding in each ct in gdf_ct_nyc 
dep_moderate_flattened_1 = dep_moderate_flattened[dep_moderate_flattened['Flooding_Category'] == 1]
dep_moderate_flattened_2 = dep_moderate_flattened[dep_moderate_flattened['Flooding_Category'] == 2]
ct_nyc['dep_moderate_1_area'] = gpd.overlay(gdf_ct_nyc, dep_moderate_flattened_1, how='intersection').groupby('GEOID')['geometry'].apply(lambda geom: geom.area.sum()).reindex(ct_nyc.index).fillna(0)
ct_nyc['dep_moderate_2_area'] = gpd.overlay(gdf_ct_nyc, dep_moderate_flattened_2, how='intersection').groupby('GEOID')['geometry'].apply(lambda geom: geom.area.sum()).reindex(ct_nyc.index).fillna(0)

ct_nyc['dep_moderate_1_frac'] = ct_nyc['dep_moderate_1_area'] / ct_nyc['area']
ct_nyc['dep_moderate_2_frac'] = ct_nyc['dep_moderate_2_area'] / ct_nyc['area']

logger.info("Merged DEP stormwater data with NYC CT shapefile.")

logger.info("\n"+ct_nyc[['dep_moderate_1_area','dep_moderate_2_area','dep_moderate_1_frac','dep_moderate_2_frac']].describe().to_string())

2024-10-23 16:51:49 - analysis-df-assembly - INFO - Merged DEP stormwater data with NYC CT shapefile.
2024-10-23 16:51:49 - analysis-df-assembly - INFO - 
       dep_moderate_1_area  dep_moderate_2_area  dep_moderate_1_frac  dep_moderate_2_frac
count          2325.000000          2325.000000          2325.000000          2325.000000
mean          33568.861141         17255.143767             0.009435             0.004953
std           77948.072724         49174.975178             0.016486             0.014127
min               0.000000             0.000000             0.000000             0.000000
25%               0.000000             0.000000             0.000000             0.000000
50%            5211.088807             0.000000             0.001501             0.000000
75%           33654.810667          9705.671683             0.012128             0.003431
max          886875.925303        660828.326813             0.150287             0.254063


In [882]:
# 311 from September 29 Flood 
nyc311_sep29 = pd.read_csv('flooding/data/nyc311_flooding_sep29.csv').dropna(subset=['latitude','longitude'])

nyc311_sep29 = gpd.GeoDataFrame(nyc311_sep29, geometry=gpd.points_from_xy(nyc311_sep29.longitude, nyc311_sep29.latitude), crs=WGS).to_crs(PROJ)

logger.info("Loaded and processed 311 data from September 29, 2023.")

2024-10-23 16:51:49 - analysis-df-assembly - INFO - Loaded and processed 311 data from September 29, 2023.


In [883]:
# for each unique val of descriptor, create a column in gdf_ct_nyc with the count of 311 calls of that descriptor type in each tract 
for descriptor in nyc311_sep29['descriptor'].unique():
    # human-readable column name 
    # remove anything inside () 
    desc = descriptor.split('(')[0].strip()
    desc = desc.lower().replace(' ', '_') + '_311c'

    gdf_ct_nyc[desc] = gdf_ct_nyc['geometry'].apply(lambda x: nyc311_sep29[nyc311_sep29['descriptor'] == descriptor].within(x).sum())

logger.info("Merged 311 data with NYC CT shapefile.")
logger.info("\n"+ gdf_ct_nyc[[c for c in gdf_ct_nyc.columns if '_311c' in c]].sum().to_string())


2024-10-23 16:51:53 - analysis-df-assembly - INFO - Merged 311 data with NYC CT shapefile.
2024-10-23 16:51:53 - analysis-df-assembly - INFO - 
sewer_backup_311c                    1081
street_flooding_311c                  625
catch_basin_clogged/flooding_311c     429
manhole_overflow_311c                  35
highway_flooding_311c                   1


In [884]:
# merge ct_nyc with gdf_ct_nyc
ct_nyc = ct_nyc.merge(gdf_ct_nyc, left_index=True, right_on='GEOID')

In [885]:
COLS_ALLOWED_NA_VALS = ['empirical_estimate']
def na_validation(df, cols_allowed_na_vals):
    for c in df.columns:
        if c in cols_allowed_na_vals:
            continue
        if df[c].isna().sum() > 0:
            logger.error(f"Column {c} has {df[c].isna().sum()} NA values.")
    else: 
        logger.success("No N/A values found in columns.")
na_validation(ct_nyc, COLS_ALLOWED_NA_VALS)

2024-10-23 16:51:53 - analysis-df-assembly - SUCCESS - No N/A values found in columns.


In [886]:
# final cleaning 
TO_DROP = ['tract_id', 'n_images_by_area_']
# drop all columns that match regex of entry in list 
current_cols = ct_nyc.columns
for c in TO_DROP:
    ct_nyc = ct_nyc.loc[:, ~ct_nyc.columns.str.contains(c)]

logger.info(f"Dropped columns: {set(current_cols) - set(ct_nyc.columns)}")

2024-10-23 16:51:53 - analysis-df-assembly - INFO - Dropped columns: {'n_images_by_area_p_alop', 'n_images_by_area_ct'}


In [887]:
todays_date = pd.to_datetime('today').strftime('%m%d%Y')
ct_nyc.to_csv(f'analysis_df_{todays_date}.csv', index=False)